In [1]:
# todo: clean up markdown, is 6.5 correct???

#### 6.1
By randomly sampling with replacement, we are building sub-samples from the sample data with slightly different distributions of the variables. Sampling without replacement would lead to higher variance.   
If we select a card from a deck, and then replace it and select a second card, the first one doesn't effect the probability of the second one so the covariance b/w the two is 0. If sampling w/o replacement, the two selections are not independent i.e. the covariance is not 0. We want the probability distribution of selecting features to acutally reflect that of the population and not overfit.  
If our dataset was infinite size, than we could sample without replacement but since this is not possible, our finite dataset will always be overfit, or higher variance if we don't sample with replacement.  
Ref: https://web.ma.utexas.edu/users/parker/sampling/repl.htm


#### 6.2
a) With low uniqueness/high overlap, bagging will be less-prone to overfitting than a single classifier, yet it will still be overfit as for example for random forest, it will be a large number of very similar trees that are all overfit. (pg 98)  
b) OOB accuracy will be largely inflated as random samlping with replacement will place samples in training set that is very similar (very similar due to financial data having lots of redundant observations) to out of bag. Hence not very reliable in financial applications (pg.97)

In [26]:
# 6.3
from sklearn.datasets import load_iris
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

In [14]:
iris = load_iris()
y = pd.Series(iris.target)
X = pd.DataFrame(iris.data)

In [59]:
# dt_clf = DecisionTreeClassifier(criterion='entropy', max_features='auto', class_weight='balanced')
dt_clf = DecisionTreeClassifier(max_features='auto')
bag_clf = BaggingClassifier(base_estimator=dt_clf, n_estimators=1000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [60]:
bag_clf.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=1000, n_jobs=None, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [61]:
y_pred = bag_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))
for (i, t),p in zip(y_test.iteritems(), y_pred):
    if t != p:
        print(i)

0.98
133


In [55]:
# rf_clf = RandomForestClassifier(n_estimators=1000, class_weight='balanced_subsample', criterion='entropy')
rf_clf = RandomForestClassifier(n_estimators=1000)

In [56]:
rf_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [57]:
y_pred = rf_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))
for (i, t),p in zip(y_test.iteritems(), y_pred):
    if t != p:
        print(i)

0.98
133


### 6.3
Predicitions above are the same so not a good example. Regardless, the main difference between a bagging classifier/ensemble of decision trees to a random forest classifier is that the random forest has a second level of randomness (first being input data randomly subsampled w/ replacement): subsampling the features at each node split (w/o replacement).  
Sklearn's deafult max_features for RandomForestClassifier sqrt(n_features) or 2 features for our 4 feature iris dataset. Therefore, if we create the decision tree classifier like so: DecisionTreeClassifier(max_features='auto'), where max_features='auto', then it too will use sqrt(n_features) or two. This should perform identical to random forest.  
Ref: https://quantdare.com/random-forest-many-are-better-than-one/  
(pg. 98)  

### 6.4
a) Suppose we want to use the minimum number of trees in a random forest to achieve some accuracy. Since increasing the number of trees will always improve predicitions (no overfitting as it's average of all trees), we can then decrease number of feautres which leads to lower correlation among tree hence increasing forest accuracy as a whole. On the other hand, increasing number of features will increase a decision trees performance. In conclusion, we need the number of features to increase as number of trees decrease.  
b) Yes, if the number of trees are too small and features too larges, then some features might be completely missed in the subspaces, which would hurt accuracy.  
c) Since additional trees never hurt random forest's predictive power, just genralizes more, then there is no harm even when number of observations is small. There might not be much benefit since the the small sample size will be generalized over and over, but the number of trees can't hurt regardless how high it is.  
Ref: https://stats.stackexchange.com/questions/36165/does-the-optimal-number-of-trees-in-a-random-forest-depend-on-the-number-of-pred  
https://datascience.stackexchange.com/questions/13323/minimum-number-of-trees-for-random-forest-classifier  
https://datascience.stackexchange.com/questions/23666/how-many-features-to-sample-using-random-forests  

### 6.5
Stratified k-fold cross-validation is cross-validation but instead of splitting randomly into k folds, we attempt to rearrange the data so it is respresentative of the whole data set (e.g. if each class comprises 50% of the data, then we want to arrange the folds so that each class is 50%). Out of bag accuracy tests data points on trees in which that sample was not used in the training of that tree. Therefore, out of bag is attempting to test how accurate individual trees are on data it has never seen while stratified k-fold cross-validation tests data that the trees have seen before.  